In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
#Importing the relevant libraries for the project

In [ ]:
def path_finder(date):
    #this function will find the day of the week according to the day year and month
    #the purpose is to be able to find what day has more activity.
    day_name = ['Lunes','Martes','Miercoles','Jueves','Viernes','Sabado','Domingo']
    date = str(date)
    day = datetime.datetime.strptime(date, '%Y-%m-%d').weekday()
    day_found = day_name[day]
    return day_found

def twitter_cleaner(mes):
    #this function will clean completely the dataset given by twitter when you download by tweet.
    dataset = pd.read_csv('{}.csv'.format(mes)) #Reading the file
    mes = str(mes)
    dataset = pd.DataFrame(dataset) #Making sure the dataset is on a pandas DataFrame object type
    dataset['hora']=pd.to_datetime(dataset['hora']) # Changing feature content type
    dataset.drop(['ID del Tweet',
 'Enlace permanente de Tweet', 'impresiones promocionado',
 'interacciones promocionado',
 'tasa de interacción promocionado',
 'retweets promocionado',
 'respuestas promocionado',
 'me gusta promocionado',
 'clics de perfil de usuario promocionado',
 'clics en URL promocionado',
 'clics de etiquetas promocionado',
 'ampliaciones de detalles promocionado',
 'clics en enlaces permanentes promocionado',
 'Se abre la aplicación promocionado',
 'descargas de app promocionado',
 'seguimientos promocionado',
 'enviar Tweet por correo electrónico promocionado',
 'marcar teléfono promocionado',
 'visualizaciones multimedia promocionado',
 'interacciones con el contenido multimedia promocionado',
 'Se abre la aplicación', 'descargas de app',
 'enviar Tweet por correo electrónico', 'marcar teléfono', 'seguimientos', 'clics en enlaces permanentes'], axis=1, inplace=True) # Dropping not relevant columns

    dataset.columns = ['Texto del Tweet', 'Hora', 'impresiones', 'interacciones',
       'tasa de interacción en %', 'retweets', 'respuestas', 'me gusta',
       'clics de perfil de usuario', 'clics en URL', 'clics de etiquetas',
       'ampliaciones de detalles', 'visualizaciones multimedia',
       'interacciones con el contenido multimedia']#changing the name of some columns
    dataset['tasa de interacción en %'] =  dataset['tasa de interacción en %'].apply(lambda x: x*100) #Engineering the columns so we can see it on percentage
    dataset['tasa de interacción en %'] = np.round(dataset['tasa de interacción en %'],2) #A simple rounding, too many decimals.

    list_to_integer = [
              'impresiones', 'interacciones','retweets', 'respuestas', 'me gusta',
       'clics de perfil de usuario', 'clics en URL', 'clics de etiquetas',
       'ampliaciones de detalles'] #Im too lazy to change those columns to integer one by one
    for name in list_to_integer:
        dataset[name] = np.int64(dataset[name])#changing the column to integer

    dataset['Fecha'] = dataset['Hora'].apply(str).copy()
    
    for i in range(0, int(dataset['Fecha'].shape[0])):
        dataset['Fecha'][i] = dataset['Fecha'][i][:10] #Fechas means date, and the actual kind of date I want is only with Year, month and day, not with those weird numbers it had.

    dataset['Dia'] = dataset['Fecha'].copy()
    dataset['Dia'] = dataset['Dia'].apply(lambda x: path_finder(x))#Implementing the path finder function previously described

    dataset['Fecha'] =pd.to_datetime(dataset['Fecha'])
    
    #dataset['Fecha'] = dataset['Fecha'].apply(pd.to_datetime)
    #dataset['hora'] = pd.to_datetime(dataset['hora'], format='%H%M')
    dataset['Hora'] = dataset['Hora'].apply(str) #Changing hora feature to string it was datetime
    e = list(dataset['Hora'].values)
    for i in range(0, len(e)):#This for will engineer the hora feature so the values will be two integers like 14, 14 hours of course.
        e[i] = e[i][11:13] 
    dataset['Hora'] = e
    dataset['Hora'] = np.int64(dataset['Hora'])
    #dataset['Hora'] = pd.Timestamp(dataset['Hora'], unit='h')
    dataset = dataset[['Texto del Tweet',
        'Fecha',
        'Dia',
        'Hora',
        'impresiones',
        'interacciones',
        'tasa de interacción en %',
        'retweets',
        'respuestas',
        'me gusta',
        'clics de perfil de usuario',
        'clics en URL',
        'clics de etiquetas',
        'ampliaciones de detalles',
        'visualizaciones multimedia',
        'interacciones con el contenido multimedia']] #making sure the features I want will be in the final result file
     
    dataset.to_csv('{}_clean.csv'.format(mes), encoding='utf-8', index=False) #Saving the engineering.
    dataset['impresiones'].describe().to_csv('{}_described_clean.csv'.format(mes), encoding='utf-8', index=True) #A quick statistical description for the numeric values and saving it.

def fb_cleaner_scope(name): #Short function to engineer the date column for facebook content data
    df = pd.read_csv('{}.csv'.format(name), encoding='latin-1')
    df['Fecha'] = df['Fecha'].apply(lambda x: x[0:10])
    df.to_csv('{}_clean.csv'.format(name), encoding='utf-8', index=False)


def fb_cleaner_content(name): #Short function to drop not relevant columns and engineering the hour columns, and saving it immediately
    df = pd.read_csv('{}.csv'.format(name), encoding='latin-1')
    df = df.drop(columns=['Resultados','Costo por resultado'])
    df['Hora de publicación'] = df['Hora de publicación'].apply(lambda x: x[0:10])
    df.to_csv('{}_clean.csv'.format(name), encoding='utf-8', index=False)
    df['Alcance'].describe().to_csv('{}_described.csv'.format(name))


In [ ]:
fb_cleaner_content('Contenido feb final fb')
twitter_cleaner('twitter feb')
fb_cleaner_scope('Tendencias Feb fb')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
